### 1. Softmax regression (multi-class logistic regression):

In [57]:
import warnings 

import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
tf.compat.v1.disable_eager_execution()


#### 1.1. Read in the data:

In [58]:
# We will use Iris data.
# 4 explanatory variables.
# 3 classes for the response variable.
data_raw = load_iris()
data_raw.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [59]:
# Print out the description.
# print(data_raw['DESCR'])

In [60]:
X = data_raw['data']
y = data_raw['target']

In [61]:
# Check the shape.
print(X.shape)
print(y.shape)

(150, 4)
(150,)


#### 1.2. Data pre-processing:

In [62]:
# One-Hot-Encoding.
y = np.array(pd.get_dummies(y, drop_first=False))               # drop_frist = False for one-hot-encoding.
y.shape

(150, 3)

In [63]:
# Scaling
X = scale(X)


In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)
n_train_size = y_train.shape[0]

#### 1.3. Do the necessary definitions:

In [65]:
batch_size = 100                                # Size of each (mini) batch.
n_epochs  = 30000                               # Number of epochs.
learn_rate = 0.05

define the weights and bias , we have to make sure to set the approriate weights , for the sake of simplicity we fill them with ones , that means you use ``tf.ones()``

In [66]:
W = tf.Variable(tf.ones([4,3]))                 # Initial value of the weights = 1.
b = tf.Variable(tf.ones([3]))                   # Initial value of the bias = 1.


define placeholders for x and y 

In [67]:
# Number of rows not specified. Number of columns = numbmer of X variables please use 4.
X_ph =tf.compat.v1.placeholder(tf.float32, shape=([None,4]))
# Number of rows not specified. Number of columns = number of classes of the y variable = 3.
y_ph = tf.compat.v1.placeholder(tf.float32, shape=([None,3]))


define the model , recall that $y\_model = X*W + b$

In [68]:
# Model.
# Not strictly necessary to apply the softmax activation. => in the end we will apply argmax() function to predict the label!
# y_model = tf.nn.softmax(tf.matmul(X_ph, W) + b)
# The following should contain what is inside the softmax function
y_model = tf.nn.softmax(tf.matmul(X_ph, W) + b)


defin the loss function , recall that we are dealing with a multi-class classification problem ,thus we will need to work with softmax and cross_entropy (hint : use ``tf.nn.softmax_cross_entropy_with_logits_v2`` )

In [69]:
loss  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ph, logits=y_model))     # Loss = cross entropy. 

define your optimizer , use gradient descent

In [70]:
optimizer = tf.compat.v1.train.MomentumOptimizer(
    learning_rate=0.001, momentum=0.9)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer()


#### 1.4. Training and Testing:

- train your model this time using batches , (hint : make sure to use slicing and  ``np.random.choice`` to get a sample of index with a size equal to ``batch_size``.
- since we are working with batches ,at each iteration the my_feed dictionary will change according to batches


In [78]:
with tf.Session() as sess:
        # Variables initialization.
        sess.run(init)
        # Training.
        for i in range(n_epochs):
            #define your batches here in two line of codes #
            index = np.random.choice(len(X_train), batch_size)
            x_b = []
            y_b = []
            for j in index:
                x_b.append(X_train[j])
                y_b.append(y_train[j])
            my_feed = {X_ph: x_b, y_ph: y_b}
            #run training using sess.run as usual#
            sess.run(train, feed_dict=my_feed)
            if (i + 1) % 2000 == 0:
                   # Print the step number at every multiple of 2000.
                   print("Step : {}".format(i + 1))
        # Testing.
        # In argmax(), axis=1 means horizontal direction.
        correct_predictions = tf.equal(tf.argmax(y_ph, axis=1), tf.argmax(y_model, axis=1))
        # Recast the Boolean as float32 first. Then calculate the mean.
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
        accuracy_value = sess.run(accuracy, feed_dict={X_ph: X_test, y_ph: y_test})             # Actually run the test with the test data.


Step : 2000
Step : 4000
Step : 6000
Step : 8000
Step : 10000
Step : 12000
Step : 14000
Step : 16000
Step : 18000
Step : 20000
Step : 22000
Step : 24000
Step : 26000
Step : 28000
Step : 30000


Print the testing result.

In [79]:
print("Accuracy = {:5.3f}".format(accuracy_value))

Accuracy = 0.933
